In [ ]:
import nltk
import random
from nltk.corpus import movie_reviews as mr
from sklearn.feature_extraction.text import CountVectorizer

Mirar como es el fichero:

In [ ]:
print(len(mr.fileids()) )#hay 2000 ficheros (1000 exemples positius i 1000 negatius)
print(mr.words('pos/cv000_29590.txt'))
print(mr.categories())
print(mr.fileids('pos')) #lista todos los ficheros positivos
print((list(mr.words('pos/cv000_29590.txt')), 'pos') )

# Particion Train y Test:

In [ ]:
doc_pos = [mr.words(f) for f in mr.fileids('pos')]
doc_neg = [mr.words(f) for f in mr.fileids('neg')]

In [ ]:
random.shuffle(doc_pos)
random.shuffle(doc_neg)

In [ ]:
train_pos = doc_pos[:int((len(doc_pos)*0.7))] #70% para el train
test_pos = doc_pos[int((len(doc_pos)*0.7)):] #30% final para el test

train_neg = doc_neg[:int((len(doc_neg)*0.7))]
test_neg = doc_neg[int((len(doc_neg)*0.7)):]

In [ ]:
train = train_pos + train_neg
test = test_pos + test_neg
#hay que guardar estos ficheros porque cada vez que ejecutemos el codigo el random va a cambiar el contenido

In [ ]:
print(len(train_pos), len(train_neg))
print(len(test_pos), len(test_neg))

In [ ]:
train_labels = [1]*700 + [0]*700
print(len(train_labels))
test_labels = [1]*300 + [0]*300

# Preprocesamiento

In [ ]:
import re

def preproces(text):
    text_clean = []
    # Expresión regular para eliminar todo lo que no sea una letra o un espacio
    regex = r"[^a-zA-Z\s]"
    
    # Diccionario de reemplazo
    replacements = {
        "t": "not",
        's': 'is', #aunque tambien puede ser posesivo
        "didn": "did",
        "haven": "have",
        "hasn": "has",
        "hadn": "had",
        "shouldn": "should",
        "wouldn": "would",
        "couldn": "could",
        "mustn": "must",
        'doesn': 'does',
        'isn' : 'is',
        'aren' : 'are',
        'll': 'will',
        've' : 'have', 
        'd' : 'would',
        'm': 'am'
    }
    
    for w in text:
        # Eliminar caracteres no deseados y números
        limpio = re.sub(regex, "", w)
        # Convertir a minúsculas
        limpio = limpio.lower()
        # Eliminar espacios en blanco al principio y al final
        limpio = limpio.strip()
        # Eliminar si la palabra es "s"
        #if limpio == "s":
           # continue
        # Reemplazar si está en el diccionario
        if limpio in replacements:
            limpio = replacements[limpio]
        # Filtrar tokens vacíos
        if limpio != "":
            text_clean.append(limpio)
    
    return text_clean

In [ ]:
clean_train = [preproces(words) for words in train]
print(clean_train[0])
clean_test = [preproces(words)for words in test]

# Lematizacion

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
#Poner tags y no incluir stop Words ya que no aportaran informacion importante para el Sentiment Analisys
train_tagged = []
tags = set()
stop_tags = {'DT', 'IN', 'CC', 'PRP', 'PRP$', 'WP', 'WP$', 'RP', 'TO', 'CD', 'EX', 'WDT'}
for l in clean_train:
    aux = []
    con_tag = nltk.pos_tag(l)
    for w, tag in con_tag:
        if tag not in stop_tags:
            aux.append((w, tag))
            tags.add(tag)
    train_tagged.append(aux)

In [ ]:
print(tags) #todos los tags unicos que aparecen en el train

In [ ]:
#ejemplo del resultado
print(train_tagged[0])
print(train[0]) 

In [ ]:
#ver otros ejemplos de palabras con un cierto tag
palabras = []
for l in train_tagged:
    for palabra, etiqueta in l:
            if etiqueta == '$':
                palabras.append(palabra)
print(palabras)

El tag '$' es un error del tagger pero no upondrá ningún problema. Además, al observar el tag "FW" (palabras que se etiquetan como extranjeras) vemos que podríamos eliminar errores, palabras en otros idiomas, nombres propios, pero también palabras normales. Como posteriormnte se va a hacer filtrado de frequencias esperamos que se eliminarán las palabras no deseadas y quedarán las que sí que tienen sentido.

Se lematizan solo estas categorias, reconocidas por SentiWordNet:

Sustantivos ('n')	'NN', 'NNS', 'NNP', 'NNPS'
Verbos ('v')	'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
Adjetivos ('a')	'JJ', 'JJR', 'JJS'
Adverbios ('r')	'RB', 'RBR', 'RBS'

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
wnl = nltk.stem.WordNetLemmatizer()
def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 'NNP' : 'n', 'NNPS':'n', #sustantivos
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a',  #adjetivos 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 'MD':'v', #verbos
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}  #adverbios
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

In [ ]:
train_lemat = []
for l in train_tagged:
    aux = []
    for i in l:
        aux.append(lemmatize(i))
    train_lemat.append(aux)

## Vectroización BoW

In [ ]:
vectorizer = CountVectorizer()
train_aplanado = [' '.join(sublista) for sublista in train_lemat]
X_train = vectorizer.fit_transform(train_aplanado)

In [ ]:
X_train.shape

In [ ]:
train_aplanado[1]

Eliminacion de frequencias bajas

In [ ]:
# Sumamos las frecuencias de cada palabra en todos los documentos
word_frequencies = X_train.sum(axis=0).A1
# Obtenemos el vocabulario mapeado a índices
words = vectorizer.get_feature_names_out()
# Convertimos en un diccionario {palabra: frecuencia}
freq_dict = dict(zip(words, word_frequencies))
# Ordenamos por frecuencia ascendente
sorted_freq = sorted(freq_dict.items(), key=lambda x: x[1])
print(sorted_freq[:10])  # Muestra las 10 palabras menos frecuentes


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ordenamos las frecuencias en orden descendente
sorted_freq_values = np.array(sorted(word_frequencies, reverse=True))

# Creamos la gráfica
plt.figure(figsize=(10,5))
plt.plot(sorted_freq_values, marker="o", linestyle="-")

# Agregamos etiquetas
plt.xlabel("Palabras ordenadas por frecuencia")
plt.ylabel("Frecuencia de aparición")
plt.title("Distribución de frecuencias de palabras")
plt.yscale("log") 
plt.grid()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Histograma detallado con límite en eje X
max_freq = int(max(word_frequencies))

plt.figure(figsize=(12,6))
plt.hist(word_frequencies, bins=np.arange(1, max_freq + 1), edgecolor='black', log=True)

plt.xlabel("Frecuencia exacta")
plt.ylabel("Cantidad de palabras")
plt.title("Histograma de frecuencias de palabras (zoom 0–100)")
plt.grid(axis='y')

plt.tight_layout()
plt.show()


In [ ]:
max_freq = int(max(word_frequencies))

plt.figure(figsize=(12,6))
plt.hist(word_frequencies, bins=np.arange(1, max_freq + 1), edgecolor='black', log=True)

plt.xlabel("Frecuencia exacta")
plt.ylabel("Cantidad de palabras")
plt.title("Histograma de frecuencias de palabras (zoom 0–100)")
plt.grid(axis='y')

# 🔍 Zoom en el eje X
plt.xlim(0, 50)

plt.tight_layout()
plt.show()

In [ ]:
min_freq = 3  # Umbral mínimo (si se pone mas agrande se eliminaran mas de la mitad de las observaciones)
vectorizer = CountVectorizer(min_df=min_freq)
X_filtered = vectorizer.fit_transform(train_aplanado)

In [ ]:
word_frequencies = X_filtered.sum(axis=0).A1
sorted_freq_values = np.array(sorted(word_frequencies, reverse=True))

# Creamos la gráfica
plt.figure(figsize=(10,5))
plt.plot(sorted_freq_values, marker="o", linestyle="-")

# Agregamos etiquetas
plt.xlabel("Palabras ordenadas por frecuencia")
plt.ylabel("Frecuencia de aparición filtarada")
plt.title("Distribución de frecuencias de palabras")
plt.yscale("log") 
plt.grid()

In [ ]:
X_filtered.shape #train final

# Modelos supervisados. Parte 1:

Una vez que ya tengamos los datos preprocesados de manera que faciliten a los modelos a classificar los documentos, empezaremos a construir los modelos supervisados. 

Para esta práctica hemos querido comparar tres modelos supervisados significativamente diferentes entre ellos: KNN, Random Forest y SVM (Support Vector Machine). Para cada uno de los modelos se contemplará el mismo procedimiento de entrenamiento. 

Primero de todo, se usará la función **GridSearchCV** para encontrar los mejores parámetros que se ajusten a los datos del train, posteriormente se aplicará la función cross_validate

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pandas as pd

# Tus datos
X = train_aplanado
y = train_labels

# Métricas que quieres evaluar
scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'precision_macro': 'precision_macro',
    'recall_macro': 'recall_macro'
}

# Resultados globales
resultados = {}

# ----------- MODELO 1: KNN -----------
pipe_knn = Pipeline([
    ('vectorizer', CountVectorizer(min_df=3)),
    ('clf', KNeighborsClassifier())
])

param_knn = {
    'clf__n_neighbors': [3, 5, 7]
}

grid_knn = GridSearchCV(pipe_knn, param_grid=param_knn, cv=5, scoring='accuracy')
grid_knn.fit(X, y)

mejor_knn = grid_knn.best_estimator_

# ----------- MODELO 2: Random Forest -----------
pipe_rf = Pipeline([
    ('vectorizer', CountVectorizer(min_df=3)),
    ('clf', RandomForestClassifier(random_state=42))
])

param_rf = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [None, 10, 100]
}

grid_rf = GridSearchCV(pipe_rf, param_grid=param_rf, cv=5, scoring='accuracy')
grid_rf.fit(X, y)
mejor_rf = grid_rf.best_estimator_

# ----------- MODELO 3: SVM -----------
pipe_svm = Pipeline([
    ('vectorizer', CountVectorizer(min_df=3)),
    ('clf', SVC())
])

param_svm = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf']
}

grid_svm = GridSearchCV(pipe_svm, param_grid=param_svm, cv=5, scoring='accuracy')
grid_svm.fit(X, y)
mejor_svm = grid_svm.best_estimator_



In [ ]:

# Diccionario de modelos a comparar
modelos = {
    "KNN": mejor_knn,
    "RandomForest": mejor_rf,
    "SVM": mejor_svm
}

# Guardamos resultados
resultados = {}

# mejor_knn, mejor_rf, mejor_svm son Pipelines que contienen el mejor modelo y el vectorizador. 
for nombre, modelo in modelos.items():
    cv_result = cross_validate(modelo, X, y, cv=5, scoring=scoring)

    resumen = {métrica: round(cv_result[f'test_{métrica}'].mean(), 4) for métrica in scoring}
    resultados[nombre] = resumen


# Mostrar como DataFrame ordenado
df_resultados = pd.DataFrame(resultados).T
print("🔍 Los mejores resultados de cada modelo:")
print(df_resultados)


# Modelos no supervisados. Parte 2:

In [ ]:
from collections import Counter
from textserver import TextServer
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn


In [ ]:
#filtar train_lemat

word_freq = Counter()
for doc in train_lemat:
    word_freq.update(doc)

min_freq = 5
filtered_words = {word for word, freq in word_freq.items() if freq < min_freq}
filtered_train_lemat = [
    [word for word in doc if word not in filtered_words]
    for doc in train_lemat]

In [ ]:
 
ts = TextServer('taisiia.prymak', '', 'senses') 

In [ ]:
result = ts.senses('lively')[0][0]
print(result)
print(lesk(train_lemat[8], 'lively'))
#hay palabras que Freeling es incapaz de encontrar su synset, pero lesk si

Si Freeling no encuentra el synset intentaremos encontrarlo con Lesk:

In [ ]:
def get_synsets(w, contexto):
    try:
        result = ts.senses(w)[0][0]
        categ = result[4][-1]
        id = int(result[4][0:8])
        synset1 = wn.synset_from_pos_and_offset(categ, id)
        return synset1
    except:
        return lesk(contexto, w) 

In [ ]:
len(filtered_train_lemat[1])

In [ ]:
prueba = [filtered_train_lemat[0]]
print(prueba)
etiquetas_predichas = []

## Metodo simple: Clasificaci´on de polaridad: score(s) = poss − neg

Asignación de valores neutros

Enfoque: Asignar un valor neutral (0) a todos los synsets no encontrados.

Justificación académica: Este enfoque asume que las palabras ausentes no contribuyen al sentimiento del texto. Puedes argumentar que SentiWordNet se enfoca en palabras con carga emocional clara, y por tanto, las palabras ausentes probablemente sean neutras en términos de sentimiento.

Cita potencial: "La ausencia de ciertos términos en léxicos de sentimiento como SentiWordNet puede interpretarse como evidencia de su naturaleza semánticamente neutra en contextos evaluativos" (Baccianella et al., 2010).

In [ ]:

#else: si el sinset NO esta en sentiWordNet se va a suponer que su puntuacion es 0 JUSTIFICAR 
#si es neutral clasificar como positivo?????????????????????????????????????????????????????????????????????

def predecir_sentimiento(lista_docs):

    for doc in lista_docs:
        pos_s = 0
        neg_s = 0

        for w in doc:
            s1 = get_synsets(w, doc)
            if s1 is not None: #si ni freeling ni lesk ha encontrado el syntet la funcion get_synsets(w, doc) devuelve None
                try:
                    senti_synset = swn.senti_synset(s1.name())
                    pos_s += senti_synset.pos_score()
                    neg_s += senti_synset.neg_score()
                except:
                    pass  # si no está en SentiWordNet
        etiquetas_predichas.append(1 if pos_s - neg_s >= 0 else 0)

    return etiquetas_predichas

In [ ]:
from tqdm import tqdm

for l in tqdm(filtered_train_lemat):
    ...


In [ ]:
# Define etiquetas útiles para sentimiento
POS_RELEVANTES = {'JJ', 'JJR', 'JJS',  # Adjetivos
                  'NN', 'NNS', 'NNP', 'NNPS',  # Sustantivos
                  'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verbos
                  'RB', 'RBR', 'RBS'}  # Adverbios

SOLO_ADJETIVOS = {'JJ', 'JJR', 'JJS'}

def filtrar_tagged_por_pos(tagged_docs, etiquetas_validas):
    return [[w for w, tag in doc if tag in etiquetas_validas] for doc in tagged_docs]

In [47]:
docs_relevantes = filtrar_tagged_por_pos(train_tagged, POS_RELEVANTES)
print(sum(len(doc) for doc in docs_relevantes))
preds_relevantes = predecir_sentimiento(docs_relevantes)

568794


In [ ]:
docs_adjetivos = filtrar_tagged_por_pos(train_tagged, SOLO_ADJETIVOS)
preds_adjetivos = predecir_sentimiento(docs_adjetivos)

In [ ]:
from sklearn.metrics import classification_report

print("🔍 Evaluación con POS relevantes:")
print(classification_report(train_labels, preds_relevantes))

print("\n🔍 Evaluación con solo adjetivos:")
print(classification_report(train_labels, preds_adjetivos))


In [48]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.wsd import lesk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import numpy as np
import pandas as pd

# POS relevantes
POS_TAGS = {
    'adj': {'JJ', 'JJR', 'JJS'},
    'verb': {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
    'adv': {'RB', 'RBR', 'RBS'},
    'noun': {'NN', 'NNS', 'NNP', 'NNPS'}
}

def extract_pos_sentiment_features(tagged_docs):
    X = []
    for doc in tagged_docs:
        scores = {'adj': 0, 'verb': 0, 'adv': 0, 'noun': 0}
        tokens = [w for w, _ in doc]

        for w, tag in doc:
            pos_category = None
            for key, tag_set in POS_TAGS.items():
                if tag in tag_set:
                    pos_category = key
                    break
            if pos_category:
                syn = lesk(tokens, w)
                if syn:
                    try:
                        senti = swn.senti_synset(syn.name())
                        scores[pos_category] += senti.pos_score() - senti.neg_score()
                    except:
                        pass
        X.append([scores['adj'], scores['verb'], scores['adv'], scores['noun']])
    return np.array(X)

# Extraemos features
X_features = extract_pos_sentiment_features(train_tagged)
y = train_labels

# Métricas a evaluar
scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'precision_macro': 'precision_macro',
    'recall_macro': 'recall_macro'
}

# Modelos
modelos = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Resultados
resultados = {}

for nombre, modelo in modelos.items():
    print(f"🧪 Evaluando {nombre}...")
    cv_result = cross_validate(modelo, X_features, y, cv=5, scoring=scoring)
    resumen = {métrica: round(np.mean(cv_result[f'test_{métrica}']), 4) for métrica in scoring}
    resultados[nombre] = resumen

# Mostrar tabla de resultados
df_resultados = pd.DataFrame(resultados).T
print("\n📊 Resultados promedio con cross-validation:")
print(df_resultados)

# ENTRENAR FINAL para mostrar coeficientes / importancias
print("\n🎯 Pesos finales entrenando con todo el dataset:")

lr_model = modelos["LogisticRegression"].fit(X_features, y)
rf_model = modelos["RandomForest"].fit(X_features, y)

# Pesos LR
print("\nCoeficientes (LogisticRegression):")
for i, pos in enumerate(['adj', 'verb', 'adv', 'noun']):
    print(f"{pos}: {lr_model.coef_[0][i]:.4f}")

# Importancias RF
print("\nImportancia por feature (RandomForest):")
for i, pos in enumerate(['adj', 'verb', 'adv', 'noun']):
    print(f"{pos}: {rf_model.feature_importances_[i]:.4f}")


🧪 Evaluando LogisticRegression...
🧪 Evaluando RandomForest...

📊 Resultados promedio con cross-validation:
                    accuracy  f1_macro  precision_macro  recall_macro
LogisticRegression    0.6543    0.6540           0.6547        0.6543
RandomForest          0.6371    0.6361           0.6390        0.6371

🎯 Pesos finales entrenando con todo el dataset:

Coeficientes (LogisticRegression):
adj: 0.1895
verb: 0.1000
adv: 0.1356
noun: 0.0084

Importancia por feature (RandomForest):
adj: 0.3100
verb: 0.2252
adv: 0.2365
noun: 0.2283
